# Settings

In [1]:
# Load Reticulate function
Sys.setenv(RETICULATE_PYTHON="/home/luca/anaconda3/envs/reticulate/bin/python")
library(reticulate)
reticulate::use_python("/home/luca/anaconda3/envs/reticulate/bin/python")
reticulate::use_condaenv("/home/luca/anaconda3/envs/reticulate")
reticulate::py_module_available(module='anndata') #needs to be TRUE
reticulate::import('anndata') #good to make sure this doesn't error
reticulate::py_module_available(module='leidenalg') #needs to be TRUE
reticulate::import('leidenalg') #good to make sure this doesn't error

[1] TRUE

Module(anndata)

[1] TRUE

Module(leidenalg)

In [2]:
## Patch for annotations in R4.1
# BiocManager::install("Bioconductor/GenomeInfoDb",lib = "/home/luca/R/x86_64-pc-linux-gnu-library/4.1",force = TRUE)
# library(GenomeInfoDb,lib.loc="/home/luca/R/x86_64-pc-linux-gnu-library/4.1")

In [3]:
# Load packages
pacman::p_load(dplyr, stringr, data.table, tidyr, data.table, Matrix, future, 
               hdf5r, Seurat, Signac,harmony, knitr, SoupX, 
               logr, parallel, 
               ggplot2, ggpubr, ggrepel, ggbreak, gridExtra, patchwork, grid, ggh4x)

In [5]:
# Set options
options(stringsAsFactors = FALSE)
warnLevel <- getOption('warn')
options(warn = -1)
opts_chunk$set(tidy=TRUE)

# set Future
plan("multicore", workers = 4)
# set RAM treshold
## 1000 = 1gb
RAM.tresh = 10000 * 1024^2
options(future.globals.maxSize = RAM.tresh)

In [7]:
# Set directories
assets.dir = "/nfs/lab/projects/COVID_mouse/assets/"
reference.dir = "/nfs/lab/projects/COVID_mouse/assets/BoneMarrowReference.rds"
snRNAseq.dir = "/nfs/lab/projects/COVID_mouse/seurat/5_clean.map/snRNAseq_mrg.clean.scTyped.rds"

reference.map.dir = "/nfs/lab/projects/COVID_mouse/5_clean.map/"
counts.dir = paste0(reference.map.dir, "Downstream_Files/RNA/perdonor/COUNTS/")

In [8]:
# Load markers list
cell.markers = read.table(paste(assets.dir, "Cell.markers.txt", sep = ""), sep = "\t", header = TRUE)
# Make it long, remove useless column and void markers
cell.markers <- cell.markers %>% gather(Key, marker, c(3:ncol(cell.markers)))
cell.markers = cell.markers[,-3]
cell.markers = cell.markers[cell.markers$marker != "", ]
compartment.ls = unique(cell.markers$Compartment)
celltype.ls = unique(cell.markers$Celltype)
# Factorize columns
cell.markers$Compartment = factor(cell.markers$Compartment, 
                        levels = compartment.ls)
cell.markers$CellType = factor(cell.markers$Celltype,
                        levels = celltype.ls)

In [10]:
log_open(file_name = paste0(reference.map.dir, "RNA_DownstreamFiles.log"))

[1] "/nfs/lab/projects/COVID_mouse/5_clean.map/log/RNA_DownstreamFiles.log"

# Load assay

In [12]:
log_print(" Loading data")
adata = readRDS(paste(reference.map.dir, "snRNAseq_mrg.clean.refMapped.rds", sep = ""))

[1] " Loading data"


# Cell count matrix  - RNA

In [19]:
# adding a column for formatting reasons
adata$donor = adata$library
adata$celltypes = gsub("[ /]", "_", adata$predicted.celltypes)


samples = as.character(unique(adata$donor))
samples

[1] "1_GFP1"  "10_G1C1" "11_G1C2" "12_G1C3" "2_GFP2"  "3_GFP3"  "4_RBD1" 
[8] "5_RBD2"  "6_RBD3"

In [20]:
######## SET TO WHATEVER YOUR ASSIGNMENTS ARE STORED UNDER ########
Idents(object = adata) <- "celltypes"
head(Idents(adata))
#### OUTPUT DIRECTORY #####
outdir = counts.dir

#pull out list of all cell types, removing ignore
unique_cell_types <- unique(adata$celltypes)
#unique_cell_types <- unique_cell_types[-c(11)]
print(unique_cell_types)


sample_bcs <- list()
for (sample in samples){
    sample_bcs[[sample]] <- row.names(adata[[]][adata[[]]$donor == sample,])
}

##############
#### SET TO WHATEVER ASSAY YOU WANT TO USE ######
DefaultAssay(adata) <- 'RNA'
gex.counts <- GetAssayData(adata, slot='counts')
dim(gex.counts)
head(gex.counts)
adata_matrices <- adata

1_GFP1_AAACCCAAGCGACATG-1 1_GFP1_AAACCCAAGCTCCACG-1 1_GFP1_AAACCCAAGGGCAATC-1 
              Neutrophils           Dendritic_cells               Neutrophils 
1_GFP1_AAACCCAAGTTGCCTA-1 1_GFP1_AAACCCACACGTGTGC-1 1_GFP1_AAACCCAGTCCAAAGG-1 
              Neutrophils               Neutrophils                 Monocytes 
17 Levels: Neutrophils Dendritic_cells Monocytes pro-B ... Mk_prog.

 [1] "Neutrophils"     "Dendritic_cells" "Monocytes"       "pro-B"          
 [5] "small_pre-B."    "NK_cells"        "B_cell"          "LMPPs"          
 [9] "Eo_Baso_prog."   "large_pre-B."    "Mono_prog."      "T_cells"        
[13] "Gran_Mono_prog." "Ery_Mk_prog."    "Neutro_prog."    "Erythroblasts"  
[17] "Mk_prog."       


[1]  32285 115388

  [[ suppressing 34 column names '1_GFP1_AAACCCAAGCGACATG-1', '1_GFP1_AAACCCAAGCTCCACG-1', '1_GFP1_AAACCCAAGGGCAATC-1' ... ]]



6 x 115388 sparse Matrix of class "dgCMatrix"
                                                                           
Xkr4    . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
Gm1992  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
Gm19938 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
Gm37381 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
Rp1     . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
Sox17   . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
              
Xkr4    ......
Gm1992  ......
Gm19938 ......
Gm37381 ......
Rp1     ......
Sox17   ......

 .....suppressing 115354 columns in show(); maybe adjust options(max.print=, width=)
 ..............................

In [22]:
#looping through cell types by making ^ into a function
get_per_sample_gex_SUMS <- function(cell.type, filename){
    print(paste(cell.type,Sys.time()))

    #pull out rows of gex.counts where BC Ident matches cell.type
    bcs <- names(Idents(adata_matrices)[Idents(adata_matrices) == cell.type])
    counts <- gex.counts[,colnames(gex.counts) %in% bcs]
    print(dim(counts))

    #initialize the matrix of sample gex
    counts.df <- as.data.frame(rep(0,length(row.names(gex.counts))))
    row.names(counts.df) <- row.names(gex.counts)
    colnames(counts.df) <- c('temp')

    #go through samples and calculate sum of gex values
    for (sample in samples){
        sample_cols <- colnames(counts) %in% sample_bcs[[sample]]
        counts.cut <- counts[,sample_cols]
        
        #if only one bc, this becomes a vector which is an issue
        if (typeof(counts.cut) == 'double'){
            mean.counts <- counts.cut
        #if there are NO bcs, this will return NA (just return 0 for everything)
        } else if(length(colnames(counts.cut)) == 0){
            mean.counts <- rep(0,length(row.names(counts)))
        } else {
            mean.counts <- rowSums(counts.cut)
        }
        counts.df <- cbind(counts.df,as.data.frame(mean.counts))
     }
    fin.counts.df <- counts.df[,-c(1)]
    colnames(fin.counts.df) <- samples
    head(fin.counts.df)

    #export df
    write.table(fin.counts.df, filename, sep='\t',quote=FALSE)
}

In [23]:
##### NAME YOUR FILES #####
for (cell.type in unique_cell_types){
    filename <- paste(counts.dir, cell.type, '_perdonor.gex_SoupX.RNA.counts', sep = "")
    get_per_sample_gex_SUMS(cell.type, filename)
}

[1] "Neutrophils 2024-02-08 14:39:58.286389"
[1] 32285 57783
[1] "Dendritic_cells 2024-02-08 14:40:02.362863"
[1] 32285  8943
[1] "Monocytes 2024-02-08 14:40:04.332531"
[1] 32285 21987
[1] "pro-B 2024-02-08 14:40:07.543163"
[1] 32285  1974
[1] "small_pre-B. 2024-02-08 14:40:08.470427"
[1] 32285 11136
[1] "NK_cells 2024-02-08 14:40:09.993713"
[1] 32285  4511
[1] "B_cell 2024-02-08 14:40:11.097223"
[1] 32285  5948
[1] "LMPPs 2024-02-08 14:40:12.245268"
[1] 32285   848
[1] "Eo_Baso_prog. 2024-02-08 14:40:13.15708"
[1] 32285   347
[1] "large_pre-B. 2024-02-08 14:40:13.91938"
[1] 32285   696
[1] "Mono_prog. 2024-02-08 14:40:14.774152"
[1] 32285   266
[1] "T_cells 2024-02-08 14:40:15.592236"
[1] 32285   196
[1] "Gran_Mono_prog. 2024-02-08 14:40:16.319284"
[1] 32285   265
[1] "Ery_Mk_prog. 2024-02-08 14:40:17.154137"
[1] 32285    65
[1] "Neutro_prog. 2024-02-08 14:40:17.864556"
[1] 32285   350
[1] "Erythroblasts 2024-02-08 14:40:19.181078"
[1] 32285    52
[1] "Mk_prog. 2024-02-08 14:40:19.875

In [24]:
#looping through cell types by making ^ into a function
get_gex_SUMS <- function(cell.type, filename){
    print(paste(cell.type,Sys.time()))

    #pull out rows of gex.counts where BC Ident matches cell.type
    bcs <- names(Idents(adata_matrices)[Idents(adata_matrices) == cell.type])
    counts <- gex.counts[,colnames(gex.counts) %in% bcs]
    print(dim(counts))

    #grab and sum counts
    counts.df <- as.data.frame(rep(0,length(row.names(gex.counts))))
    row.names(counts.df) <- row.names(gex.counts)
    colnames(counts.df) <- c('counts')
    counts.df$counts = rowSums(counts)
    write.table(counts.df, filename, sep='\t',quote=FALSE)
}

In [25]:
##### NAME YOUR FILES #####
for (cell.type in unique_cell_types){
    filename <- paste(counts.dir, cell.type, '_gex_SoupX.RNA.counts', sep = "")
    get_gex_SUMS(cell.type, filename)
}

[1] "Neutrophils 2024-02-08 14:40:20.679531"
[1] 32285 57783
[1] "Dendritic_cells 2024-02-08 14:40:22.670203"
[1] 32285  8943
[1] "Monocytes 2024-02-08 14:40:23.819421"
[1] 32285 21987
[1] "pro-B 2024-02-08 14:40:25.41253"
[1] 32285  1974
[1] "small_pre-B. 2024-02-08 14:40:26.207417"
[1] 32285 11136
[1] "NK_cells 2024-02-08 14:40:27.320248"
[1] 32285  4511
[1] "B_cell 2024-02-08 14:40:28.285751"
[1] 32285  5948
[1] "LMPPs 2024-02-08 14:40:29.361836"
[1] 32285   848
[1] "Eo_Baso_prog. 2024-02-08 14:40:30.316023"
[1] 32285   347
[1] "large_pre-B. 2024-02-08 14:40:31.195131"
[1] 32285   696
[1] "Mono_prog. 2024-02-08 14:40:32.011821"
[1] 32285   266
[1] "T_cells 2024-02-08 14:40:32.975955"
[1] 32285   196
[1] "Gran_Mono_prog. 2024-02-08 14:40:33.833709"
[1] 32285   265
[1] "Ery_Mk_prog. 2024-02-08 14:40:34.719872"
[1] 32285    65
[1] "Neutro_prog. 2024-02-08 14:40:35.624763"
[1] 32285   350
[1] "Erythroblasts 2024-02-08 14:40:36.535376"
[1] 32285    52
[1] "Mk_prog. 2024-02-08 14:40:37.36

# Update metadata

In [26]:
#This block of code will add how many cells are in each cell-type for each sample. Can be used for filtering
unique_cell_types <- unique(adata@meta.data$celltypes)
length(unique_cell_types)

results <- data.frame(matrix(ncol = 15, nrow = 1))
results2 <- NULL
results3 <- NULL
for (x in samples){
    test <- adata@meta.data[which(adata@meta.data$donor == x),]
    for (y in unique_cell_types)
        results[, y] <- nrow(test[which(test$celltypes == y),])
        results2 <- results[,c(16:29)]
        results2$donor <- x
        results3 <- rbind(results3, results2)
}
head(results3, n = 2)

[1] 17

,Neutrophils,Dendritic_cells,Monocytes,pro-B,small_pre-B.,NK_cells,B_cell,LMPPs,Eo_Baso_prog.,large_pre-B.,Mono_prog.,T_cells,Gran_Mono_prog.,Ery_Mk_prog.,donor
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>
1,7800,688,1276,93,182,236,265,44,18,22,6,14,7,2,1_GFP1
2,6451,474,3269,228,1355,583,713,220,96,127,77,27,68,28,10_G1C1


In [30]:
write.table(x = results3,
            file = paste(assets.dir, "metadata_celltypes.txt", sep = ""), sep = "\t", header = TRUE)


ERROR: Error in write.table(x = results3, file = paste(assets.dir, "metadata_celltypes.txt", : unused argument (header = TRUE)


In [42]:
meta2 <- merge(meta, results3, by.y='donor')
colnames(meta2)

[1] "donor"           "Heart_chamber"   "condition"       "HF_status"      
 [5] "Diabetes_status" "Age"             "Sex"             "Height"         
 [9] "Weight"          "BMI"             "CM"              "Fibroblast"     
[13] "Endothelial"     "Macrophage"      "T-NK"            "Pericyte"       
[17] "SM"              "Adipocyte"       "Neuronal"        "Mast"           
[21] "Lymph-Endo"      "B"               "Epicardial"      "Endocardial"

In [43]:
#For continuous covariates, best to scale for model fitting
meta2$Height.scaled <- scale(meta2$Height)
meta2$Weight.scaled <- scale(meta2$Weight)
meta2$BMI.scaled <- scale(meta2$BMI)
meta2$Age.scaled <- scale(meta2$Age)

In [44]:
filename = paste(assets.dir, "multiome_metadata_V2_cellnumbers.txt", sep = "")
write.table(meta2, filename, sep='\t', quote=FALSE, col.names = TRUE)